In [1]:
import pandas as pd
import numpy as np
from math import *

### Read in the data

In [31]:
df = pd.read_csv('new_york_hotels.csv', encoding='cp1252')

In [32]:
type(df)

pandas.core.frame.DataFrame

In [33]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [8]:
type(df["name"])

pandas.core.series.Series

In [16]:
df.columns

Index(['ean_hotel_id', 'name', 'address1', 'city', 'state_province',
       'postal_code', 'latitude', 'longitude', 'star_rating', 'high_rate',
       'low_rate'],
      dtype='object')

In [23]:
df["star_rating_plus5"] = df["star_rating"] + 5

In [25]:
#df.loc[df["star_rating"] >= 4]

In [26]:
df.to_csv('new_york_hotels_2.csv')

## Benchmarking example

#### Define the normalization function

In [29]:
def normalize(df, pd_series):
    pd_series = pd_series.astype(float)

    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)
    lower_bound = avg - 2*sd
    upper_bound = avg + 2*sd

    # Collapse in the outliers
    df.loc[pd_series < lower_bound , "cutoff_rate" ] = lower_bound
    df.loc[pd_series > upper_bound , "cutoff_rate" ] = upper_bound

    # Finally, take the log
    normalized_price = np.log(df["cutoff_rate"].astype(float))
    
    return normalized_price

In [42]:
#df.loc[~df["cutoff_rate"].isna()]

#### Timing the normalization function

In [43]:
%timeit df['high_rate_normalized'] = normalize(df, df['high_rate'])

5.11 ms ± 761 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Profiling the normalization function

In [44]:
%load_ext line_profiler

In [45]:
%lprun -f normalize df['high_rate_normalized'] = normalize(df, df['high_rate'])

## Haversine definition

In [46]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

## Iterrows Haversine

In [47]:
cafe_lat = 40.671
cafe_lon = -73.985

In [49]:
type(df.iterrows())

generator

In [48]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []

for index, row in df.iterrows():
    haversine_series.append(haversine(cafe_lat, cafe_lon, row['latitude'], row['longitude']))
    
df['distance'] = haversine_series

235 ms ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
haversine_series = []

for index, row in df.iterrows():
    haversine_series.append(haversine(cafe_lat, cafe_lon, row['latitude'], row['longitude']))
    
df['distance'] = haversine_series

In [54]:
#df.head()

## Apply Haversine on rows

### Timing "apply"

In [71]:
%timeit 
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)

90.6 ms ± 7.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Profiling "apply"

In [55]:
# Haversine applied on rows
%lprun -f haversine \
df.apply(lambda row: haversine(40.671, -73.985,  row['latitude'], row['longitude']), axis=1)

## Vectorized implementation of Haversine applied on Pandas series

#### Timing vectorized implementation

In [57]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

2.51 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Profiling vectorized implementation

In [58]:
# Vectorized implementation profile
%lprun -f haversine haversine(40.671, -73.985, df['latitude'], df['longitude'])

## Vectorized implementation of Haversine applied on NumPy arrays

#### Timing vectorized implementation

In [64]:
#df.dtypes

In [66]:
df['latitude']

0       42.68751
1       42.68971
2       42.72410
3       42.65157
4       42.68873
          ...   
1626    40.97275
1627    40.95466
1628    40.92625
1629    40.95375
1630    40.97308
Name: latitude, Length: 1631, dtype: float64

In [65]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

333 µs ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [57]:
%%timeit
# Convert pandas arrays to NumPy ndarrays
np_lat = df['latitude'].values
np_lon = df['longitude'].values

7.66 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Profiling vectorized implementation

In [67]:
%lprun -f haversine df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

## Cythonize that loop

#### Load the cython extension

In [68]:
%load_ext cython

#### Run unaltered Haversine through Cython

In [69]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

#### Time it

In [70]:
%timeit df['distance'] =\
       df.apply(lambda row: haversine_cy(40.671, -73.985,\
                row['latitude'], row['longitude']), axis=1)

80.9 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Redefine Haversine with data types and C libraries

In [71]:
%%cython -a
# Haversine cythonized
from libc.math cimport sin, cos, acos, asin, sqrt

cdef deg2rad_cy(float deg):
    cdef float rad
    rad = 0.01745329252*deg
    return rad
    
cpdef haversine_cy_dtyped(float lat1, float lon1, float lat2, float lon2):
    cdef: 
        float dlon
        float dlat
        float a
        float c
        float mi
    
    lat1, lon1, lat2, lon2 = map(deg2rad_cy, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    mi = 3959 * c
    return mi


#### Time it

In [72]:
%timeit df['distance'] =\
df.apply(lambda row: haversine_cy_dtyped(40.671, -73.985,\
                              row['latitude'], row['longitude']), axis=1)

46.2 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
